# Code generation CrewAI agent
---

This notebook presents a solution to build an agent using [CrewAI](https://docs.crewai.com/introduction). CrewAI enables you to create AI teams where each agent has specific roles, tools, and goals, working together to accomplish complex tasks. 

As a part of this solution, we will build a simple code generation agent that can use some content within a knowledge base and self reflection to provide executable and correct code. 

In [1]:
# Install crew ai. For installation steps, follow the instructions here: https://docs.crewai.com/installation
!pip install 'crewai[tools]'

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import json
import yaml
import time
import boto3
import random
import logging
from globals import *
from pathlib import Path
from litellm import completion
from botocore.exceptions import ClientError
from typing import Dict, List, Any, Optional

/opt/homebrew/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [3]:
# Setup logging
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
!pygmentize globals.py

import os
# global constants
CONFIG_FILE: str = "config.yml"
MODEL_ID_TO_PROMPT_ID_MAPPING_FILE: str = ".model_id_to_prompt_id_mapping.json"

LAMBDA_DIR: str = "lambda"
CONFIG_FILE: str = "config.yml"
CODE_GEN_LAMBDA: str = "code-gen"
LAMBDA_ARN_FILE: str = ".lambda_arn"

CODE_GEN_MODEL_ID: str = "amazon.nova-lite-v1:0"
DATA_DIR: str = "knowledge"
API_SPEC_FILE: str = "openapi.json"
TITAN_TEXT_EMBED_V2: str = 'amazon.titan-embed-text-v2:0'


In [5]:
config = yaml.safe_load(Path(CONFIG_FILE).read_text())
logger.info(f"config=\n{json.dumps(config, indent=2)}")

[2025-02-03 17:38:54,627] p70833 {3811199622.py:2} INFO - config=
{
  "general": {
    "app_name": "code-gen-agent",
    "description": "Amazon Bedrock Agent for generating code for the USACO benchmark",
    "role_name": "CodeGenLambdaRole",
    "region": "us-east-1",
    "model_id": "amazon.nova-micro-v1:0",
    "agent_instructions": "Generate Python code for the USACO problems. You have access to a tool for generating the code.\n",
    "ttl": 1800
  },
  "action_group": {
    "name": "Generate_Python_code",
    "description": "Generates Python code by using foundation models"
  },
  "prompt_info": {
    "name": "USACO_{model_id}",
    "description": "Generate code for the USACO benchmark"
  },
  "prompt_templates": {
    "nova": {
      "models": [
        "us.amazon.nova-pro-v1:0",
        "us.amazon.nova-lite-v1:0",
        "us.amazon.nova-micro-v1:0"
      ],
      "text": "Please reply with a Python 3 solution to the below problem. Read the general instructions below that are app

In [6]:
# fetch the current AWS region
region = config['general']['region']
# the region to be dynamically fetched
logger.info(f"Current AWS region: {region}")
bedrock_agent = boto3.client(service_name = "bedrock-agent", region_name = region)

[2025-02-03 17:38:54,712] p70833 {4102934896.py:4} INFO - Current AWS region: us-east-1
[2025-02-03 17:38:54,737] p70833 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [7]:
role_name: str = config['general']['role_name']
account: str = boto3.client('sts').get_caller_identity()['Account']
role = f"arn:aws:iam::{account}:role/{role_name}"
logger.info(f"IAM role being used: {role}")

[2025-02-03 17:38:55,156] p70833 {3154881413.py:4} INFO - IAM role being used: arn:aws:iam::218208277580:role/CodeGenLambdaRole


## Define the prompt template to model ID mapping
---


In [8]:
import re
def create_or_update_bedrock_prompt(name: str, description: str, text: str, parameters: dict, model_id: str):
    """
    Creates a prompt configuration for Amazon Bedrock using specified parameters.
    
    This function constructs a prompt template with inference settings and template configuration
    for use with Amazon Bedrock's language models. It automatically extracts input variables 
    from mustache-style placeholders in the template text.

    Args:
        name (str): The name identifier for the prompt template
        description (str): A description of the prompt template's purpose
        text (str): The template text containing mustache-style variables (e.g., {{variable}})
        parameters (dict): Configuration parameters including:
            - max_tokens (int): Maximum number of tokens in the response
            - temperature (float): Sampling temperature for text generation
        model_id (str): The identifier of the Bedrock model to use

    Returns:
        dict: The response from the Bedrock create_prompt API call, containing the created
              prompt configuration details

    Examples:
        >>> parameters = {
        ...     'max_tokens': 100,
        ...     'temperature': 0.7
        ... }
        >>> create_or_update_bedrock_prompt(
        ...     name="test_prompt",
        ...     description="A test prompt",
        ...     text="Hello {{name}}, how are you?",
        ...     parameters=parameters,
        ...     model_id="anthropic.claude-v2"
        ... )

    Notes:
        - The function automatically creates a default variant named "default_variant"
        - Input variables are automatically extracted from {{mustache}} syntax in the template text
        - Logs the API response using the logger module
        - Requires the bedrock_agent and logger to be properly configured
    """
    
    default_variant_name = "default_variant"
    input_variables = [dict(name=k) for k in extract_mustache_keys(text)]
    variant = {
            "inferenceConfiguration": {
            "text": {
                "maxTokens": parameters['max_tokens'],
                "temperature": parameters['temperature'],
                }
            },
            "modelId": model_id,
            "name": default_variant_name,
            "templateConfiguration": {
                "text": {
                    "inputVariables": input_variables,
                    "text": text
                }
            },
            "templateType": "TEXT"
        }
    
    try:
        response = bedrock_agent.create_prompt(name=name,
                                               description=description,
                                               variants=[variant],
                                               defaultVariant=default_variant_name)
    except ClientError as e:
        logger.error(f"exception occured while creating prompt, exception={e}")
        error_code = e.response['Error']['Code']
        error_message = e.response['Error']['Message']
        
        # Check for ConflictException
        if error_code == 'ConflictException':
            logger.error(f"got {error_code} exception, error_message={error_message}, going to update the prompt")
            # in case of prompt already exists the error messages looks like this
            # Couldn't perform CreatePrompt operation. The name USACO_amazon-nova-pro-v1-0 already exists for id FPPQT96U8Y. Retry your request with a different name., going to update the prompt"
            # so we can extract the id from the error message using the following regex
            pattern = r"exists for id ([A-Z0-9]+)\."
            match = re.search(pattern, error_message)
  
            if match:
                prompt_id = match.group(1)
                logger.info(f"id for the prompt that already exists is {prompt_id}")
                response = bedrock_agent.update_prompt(promptIdentifier=prompt_id,
                                                       name=name,
                                                       description=description,
                                                       variants=[variant],
                                                       defaultVariant=default_variant_name)
                logger.info(f"response after updating prompt = {response}")
            else:
                raise
    logger.info(f"response={json.dumps(response, indent=2, default=str)}")
    return response

In [9]:
def extract_mustache_keys(text: str) -> List[str]:
    """
    Extract all mustache-style keys ({{key}}) from the input text.
    
    Args:
        text (str): Input text containing mustache syntax
        
    Returns:
        List[str]: List of extracted keys without the curly braces
    """
    # The pattern looks for:
    # \{\{ - literal {{ (escaped because { is special in regex)
    # (.*?) - any characters (non-greedy match)
    # \}\} - literal }}
    pattern = r'\{\{(.*?)\}\}'
    
    # Find all matches and extract the group inside the braces
    matches = re.findall(pattern, text)
    
    # Strip whitespace from each match
    return [match.strip() for match in matches]

In [10]:
model_id_to_prompt_mapping = {}
for model_family, info in config['prompt_templates'].items():
    for model_id in info['models']:
        name = config['prompt_info']['name'].format(model_id=model_id)
        name = re.sub('[:\.]', '-', name)
        try:
            response = create_or_update_bedrock_prompt(name,
                                                       config['prompt_info']['description'],
                                                       info['text'],
                                                       config['inference_parameters'],
                                                       model_id)
            model_id_to_prompt_mapping[model_id] = response['id']
        except Exception as e:
            logger.error(f"exception occurred while creating prompt, name={name}, exception={e}")

[2025-02-03 17:38:55,444] p70833 {2042494798.py:69} ERROR - exception occured while creating prompt, exception=An error occurred (ConflictException) when calling the CreatePrompt operation: Couldn't perform CreatePrompt operation. The name USACO_us-amazon-nova-pro-v1-0 already exists for id 1UUO1L4V1V. Retry your request with a different name.
[2025-02-03 17:38:55,445] p70833 {2042494798.py:75} ERROR - got ConflictException exception, error_message=Couldn't perform CreatePrompt operation. The name USACO_us-amazon-nova-pro-v1-0 already exists for id 1UUO1L4V1V. Retry your request with a different name., going to update the prompt
[2025-02-03 17:38:55,445] p70833 {2042494798.py:84} INFO - id for the prompt that already exists is 1UUO1L4V1V
[2025-02-03 17:38:55,609] p70833 {2042494798.py:90} INFO - response after updating prompt = {'ResponseMetadata': {'RequestId': 'b3f0cb4d-27de-4a3e-98ff-3192c654011b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 03 Feb 2025 22:38:55 GMT', 'cont

In [11]:
model_id_to_prompt_mapping
Path(MODEL_ID_TO_PROMPT_ID_MAPPING_FILE).write_text(json.dumps(model_id_to_prompt_mapping, indent=2))

261

In [12]:
model_id_to_prompt_mapping

{'us.amazon.nova-pro-v1:0': '1UUO1L4V1V',
 'us.amazon.nova-lite-v1:0': 'MVQQGV6P2D',
 'us.amazon.nova-micro-v1:0': 'NV2HFPFT98',
 'us.anthropic.claude-3-5-haiku-20241022-v1:0': 'QABCXXUEP4',
 'us.anthropic.claude-3-5-sonnet-20241022-v2:0': 'N4IIEO4HO4'}

## Define the code generation task
---

In this portion of the solution we create a `gen_code` function that generates Python code for a given problem statement. This function retrieves the prompt template via a Bedrock agent, hydrates the template, and runs inference.

In [13]:
model_id_to_prompt_mapping

{'us.amazon.nova-pro-v1:0': '1UUO1L4V1V',
 'us.amazon.nova-lite-v1:0': 'MVQQGV6P2D',
 'us.amazon.nova-micro-v1:0': 'NV2HFPFT98',
 'us.anthropic.claude-3-5-haiku-20241022-v1:0': 'QABCXXUEP4',
 'us.anthropic.claude-3-5-sonnet-20241022-v2:0': 'N4IIEO4HO4'}

In [14]:
# Sometimes the model refuses to generate content; we define a custom exception.
class NoContentGeneratedException(Exception):
    pass

# A canned failure response (if needed)
FAILED_RESPONSE = """
import sys

def main():
    input = sys.stdin.read
    data = input().split()
    print(data)

if __name__ == "__main__":
    main()
"""

# Regular expression to extract Python code blocks wrapped in ```python ... ```
REGEX_FOR_PY_CODE_EXTRACTION: str = r"```python\n(.*?)```"

def _process_task(model_name: str, formatted_prompt: str, inference_params: dict) -> str:
    """
    Runs inference for a prompt using the specified model.
    Retries (with delays and jitter) in case of errors.
    """
    max_retries: int = 10
    retry_delay: int = 60  # seconds
    logger.info(f"Running inference with prompt:\n{formatted_prompt}")
    
    for attempt in range(max_retries):
        try:
            response = completion(
                model=model_name,
                model_id=None,
                messages=[{"role": "user", "content": formatted_prompt}],
                max_tokens=inference_params["max_tokens"],
                temperature=inference_params["temperature"],
                n=inference_params["n"],
            )
            logger.info(f"Raw Response: {response}")
            
            # If the model returned no completion tokens, raise a custom exception.
            if response['usage']['completion_tokens'] == 0:
                content = response["choices"][0]["message"]["content"]
                raise NoContentGeneratedException(f"Completion tokens is 0, content={content}")
            
            return response["choices"][0]["message"]["content"]
        
        except NoContentGeneratedException as e:
            if attempt < max_retries - 1:
                this_retry_delay = retry_delay * (attempt + 1) + random.randint(1, 10)
                logger.error(f"{e}, attempt {attempt + 1}. Retrying in {this_retry_delay} seconds...")
                time.sleep(this_retry_delay)
                continue
            else:
                logger.error("Max retries exceeded for task (NoContentGeneratedException).")
                raise
                
        except RateLimitError as e:
            if attempt < max_retries - 1:
                this_retry_delay = retry_delay * (attempt + 1) + random.randint(1, 10)
                logger.error(f"{e}, attempt {attempt + 1}. Retrying in {this_retry_delay} seconds...")
                time.sleep(this_retry_delay)
                continue
            else:
                logger.error("Max retries exceeded for task (RateLimitError).")
                raise
                
        except Exception as e:
            logger.error(f"Unexpected error processing task: {str(e)}")
            raise

def hydrate_prompt(prompt_template: str, values: dict) -> str:
    """
    Renders a prompt template using Jinja2.
    """
    from jinja2 import Template
    template = Template(prompt_template)
    return template.render(values)

def gen_code(query: str, model_id: str) -> Optional[str]:
    """
    Uses Amazon Bedrock (via litellm) to generate Python code for a given problem statement.
    This function retrieves the prompt template via a Bedrock agent, hydrates the template,
    and runs inference.
    """
    # Get the prompt mapping from an environment variable
    logger.info(f"MODEL_ID_TO_PROMPT_ID_MAPPING: {model_id_to_prompt_mapping}")
    prompt_id = model_id_to_prompt_mapping.get(model_id)
    logger.info(f"Found prompt_id={prompt_id} for model_id={model_id}")
    if prompt_id is not None:
        bedrock_agent = boto3.client(service_name="bedrock-agent", region_name=os.environ.get('AWS_REGION', region))
        prompt_info = bedrock_agent.get_prompt(promptIdentifier=prompt_id)
        prompt_template = prompt_info['variants'][0]['templateConfiguration']['text']['text']
        prompt = hydrate_prompt(prompt_template, {"question": query})
        
        inference_params = prompt_info['variants'][0]['inferenceConfiguration']['text']
        # Adjust key names for litellm: change maxTokens -> max_tokens
        inference_params["max_tokens"] = inference_params.pop("maxTokens")
        inference_params["n"] = 1
        logger.info(f"Running inference for model_id={model_id} with parameters: {inference_params}")
    else:
        logger.error(f"No prompt id found for model_id={model_id}")
        return None
    
    bedrock_model_id = f"bedrock/{model_id}"
    generated_text = _process_task(bedrock_model_id, prompt, inference_params)
    return generated_text


## Create a tool and assign it as a task
---

Now, we will create a code generation tool that will wrap the code generation functionality. This tool will be registered to the crewAI agent and invoked once a user asks a question.

In [15]:
# if it is us-east-1 or us-west-2, use the inference profile model id
if "us-east-1" in region or "us-west-2" in region:
    CODE_GEN_MODEL_ID = f"us.{CODE_GEN_MODEL_ID}"
    logger.info(f"Using inference profile model id: {CODE_GEN_MODEL_ID}")

[2025-02-03 17:38:56,805] p70833 {791345060.py:4} INFO - Using inference profile model id: us.amazon.nova-lite-v1:0


In [16]:
import re
from crewai.tools import tool

@tool("code_generation_tool")
def code_generation_tool(question: str, model_id: str) -> str:
    """
    This tool generates Python code for a given USACO problem statement.
    
    It returns a solution that is wrapped in markdown code block delimiters.
    The agent will extract the code block for further processing.
    """
    # Dummy code-generation logic for demonstration.
    generated_text = gen_code(question, model_id)
    if generated_text is None:
        return "Error: Code generation failed."
    # Attempt to extract a Python code block wrapped by ```python ... ```
    regex = r"```python\n(.*?)```"
    match = re.search(regex, generated_text, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return generated_text.strip()

### Create a `JSON` knowledge store
---

Knowledge in `CrewAI` is a powerful system that allows AI agents to access and utilize external information sources during their tasks. Think of it as giving your agents a reference library they can consult while working.

In this portion of the notebook, we will create a `JSON` knowledge source that will store `JSON` API specs that the agent will have access to while generating code.

In [17]:
import shutil
from crewai.knowledge.source.json_knowledge_source import JSONKnowledgeSource

if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

# initialize an embedder
embedder = {
    "provider": "bedrock",
    "config": {
        "model": TITAN_TEXT_EMBED_V2
    },
}

try:
    if os.path.exists(DATA_DIR):
        shutil.copy(API_SPEC_FILE, os.path.join('knowledge', API_SPEC_FILE))
        api_spec = JSONKnowledgeSource(file_paths=API_SPEC_FILE, embedder=embedder)
except Exception as e:
    logger.error(f"Error creating knowledge source: {e}")

## Create the CrewAI agent

In [18]:
from crewai import LLM
# customize your own LLM
amazon_nova_llm = LLM(
    model=CODE_GEN_MODEL_ID,  
    temperature=0.1,
    max_tokens=256,
    top_p=0.9,
)

In [19]:
from crewai import Agent, Task, Crew

# Create a Code Generation Agent that registers the CodeGenerationTool.
code_gen_agent = Agent(
    role="Code Generation Agent",
    goal="Generate Python code for USACO problems.",
    backstory="An expert in code generation using Amazon Bedrock, focused on USACO challenges.",
    tools=[code_generation_tool],
    verbose=True, 
    llm=amazon_nova_llm,
    knowledge_source=[api_spec]
)

# Define a task for code generation.
code_task = Task(
    description="Generate Python code for the following USACO problem: {question} using {model_id}",
    expected_output="Python code solution for the provided USACO problem.",
    agent=code_gen_agent,
    output_file="generated_code.py"
)

In [20]:
# Crew that includes the Code Generation Agent.
crew = Crew(
    agents=[code_gen_agent],
    tasks=[code_task],
    verbose=True, 
    knowledge_source=[api_spec]
    # planning=True,
    # planning_llm=amazon_nova_llm
)

In [21]:
crew_inputs = {
    'question': 'What is the code to upload files to Amazon S3?',
    'model_id': 'us.amazon.nova-lite-v1:0'
}

crew.kickoff(inputs=crew_inputs) 

17:38:57 - LiteLLM:INFO: utils.py:2825 - 
LiteLLM completion() model= us.amazon.nova-lite-v1:0; provider = bedrock
[2025-02-03 17:38:57,603] p70833 {utils.py:2825} INFO - 
LiteLLM completion() model= us.amazon.nova-lite-v1:0; provider = bedrock
[2025-02-03 17:38:57,613] p70833 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials


# Agent: Code Generation Agent
## Task: Generate Python code for the following USACO problem: What is the code to upload files to Amazon S3? using us.amazon.nova-lite-v1:0


[2025-02-03 17:38:59,099] p70833 {_client.py:1026} INFO - HTTP Request: POST https://bedrock-runtime.us-west-2.amazonaws.com/model/us.amazon.nova-lite-v1:0/converse "HTTP/1.1 200 OK"
17:38:59 - LiteLLM:INFO: utils.py:1030 - Wrapper: Completed Call, calling success_handler
[2025-02-03 17:38:59,102] p70833 {utils.py:1030} INFO - Wrapper: Completed Call, calling success_handler
[2025-02-03 17:38:59,113] p70833 {720503723.py:88} INFO - MODEL_ID_TO_PROMPT_ID_MAPPING: {'us.amazon.nova-pro-v1:0': '1UUO1L4V1V', 'us.amazon.nova-lite-v1:0': 'MVQQGV6P2D', 'us.amazon.nova-micro-v1:0': 'NV2HFPFT98', 'us.anthropic.claude-3-5-haiku-20241022-v1:0': 'QABCXXUEP4', 'us.anthropic.claude-3-5-sonnet-20241022-v2:0': 'N4IIEO4HO4'}
[2025-02-03 17:38:59,113] p70833 {720503723.py:90} INFO - Found prompt_id=MVQQGV6P2D for model_id=us.amazon.nova-lite-v1:0
[2025-02-03 17:38:59,336] p70833 {720503723.py:101} INFO - Running inference for model_id=us.amazon.nova-lite-v1:0 with parameters: {'temperature': 0.6999999880



# Agent: Code Generation Agent
## Using tool: code_generation_tool
## Tool Input: 
"{\"question\": \"What is the code to upload files to Amazon S3? using us.amazon.nova-lite-v1:0\", \"model_id\": \"us.amazon.nova-lite-v1:0\"}"
## Tool Output: 
# Import necessary modules from the AWS SDK
import boto3
from botocore.exceptions import NoCredentialsError

def upload_to_s3(file_name, bucket, object_name=None):
    """
    Upload a file to an S3 bucket.

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """
    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Initialize the S3 client
    s3_client = boto3.client('s3')

    # Upload the file
    try:
        s3_client.upload_file(file_name, bucket, object_name)
        print(f"File {file_name} uploaded successfull

[2025-02-03 17:39:06,352] p70833 {_client.py:1026} INFO - HTTP Request: POST https://bedrock-runtime.us-west-2.amazonaws.com/model/us.amazon.nova-lite-v1:0/converse "HTTP/1.1 200 OK"
17:39:06 - LiteLLM:INFO: utils.py:1030 - Wrapper: Completed Call, calling success_handler
[2025-02-03 17:39:06,353] p70833 {utils.py:1030} INFO - Wrapper: Completed Call, calling success_handler




# Agent: Code Generation Agent
## Final Answer: 
```python
# Import necessary modules from the AWS SDK
import boto3
from botocore.exceptions import NoCredentialsError

def upload_to_s3(file_name, bucket, object_name=None):
    """
    Upload a file to an S3 bucket.

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """
    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Initialize the S3 client
    s3_client = boto3.client('s3')

    # Upload the file
    try:
        s3_client.upload_file(file_name, bucket, object_name)
        print(f"File {file_name} uploaded successfully to {bucket}/{object_name}")
        return True
    except FileNotFoundError:
        print("The file was not




CrewOutput(raw='```python\n# Import necessary modules from the AWS SDK\nimport boto3\nfrom botocore.exceptions import NoCredentialsError\n\ndef upload_to_s3(file_name, bucket, object_name=None):\n    """\n    Upload a file to an S3 bucket.\n\n    :param file_name: File to upload\n    :param bucket: Bucket to upload to\n    :param object_name: S3 object name. If not specified then file_name is used\n    :return: True if file was uploaded, else False\n    """\n    # If S3 object_name was not specified, use file_name\n    if object_name is None:\n        object_name = file_name\n\n    # Initialize the S3 client\n    s3_client = boto3.client(\'s3\')\n\n    # Upload the file\n    try:\n        s3_client.upload_file(file_name, bucket, object_name)\n        print(f"File {file_name} uploaded successfully to {bucket}/{object_name}")\n        return True\n    except FileNotFoundError:\n        print("The file was not', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Generate